In [42]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [43]:
merged_df_accepted_copy = pd.read_csv('../../data/features/merged_df_accepted_copy.csv')

In [44]:
# drop null values
merged_df_accepted_copy = merged_df_accepted_copy.dropna()

# Rename the column Trip ID to order_id
# merged_df_accepted_copy = merged_df_accepted_copy.rename(columns={'Trip ID': 'order_id'})

In [45]:
# get the first 10 rows
merged_df_accepted_copy = merged_df_accepted_copy.head(500)

In [46]:
# save the df to csv file
merged_df_accepted_copy.to_csv('../../data/features/merged_df_accepted_copy.csv', index=False)

In [29]:

# 1. Split Data into Training and Hold-out Set
train_data, holdout_data = train_test_split(merged_df_accepted_copy, test_size=0.2, random_state=42)


In [30]:
train_data.head()

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,Trip Origin_longitude,Trip Destination_latitude,Trip Destination_longitude,trip_duration,is_holiday,is_weekend,Trip Start Date
249,392489,"6.485481425527273,3.390712543831626","6.4522780365897425,3.5666327651395022",2021-07-01,2021-07-01 10:18:21,Thursday,9,1,July,6.485481,3.390713,6.452278,3.566633,2957.0,0,0,2021-07-01
433,392799,"6.6266919,3.3520798","6.502206,3.3050823",2021-07-01,2021-07-01 10:49:38,Thursday,9,1,July,6.626692,3.352080,6.502206,3.305082,3563.0,0,0,2021-07-01
19,392036,"6.4408899,3.5030541","6.443346099999999,3.4827817",2021-07-01,2021-07-01 08:47:05,Thursday,7,1,July,6.440890,3.503054,6.443346,3.482782,3515.0,0,0,2021-07-01
322,392612,"6.457306599999999,3.5305005","6.594897787314384,3.3409857857377805",2021-07-01,2021-07-01 11:52:23,Thursday,9,1,July,6.457307,3.530501,6.594898,3.340986,8595.0,0,0,2021-07-01
332,392626,"6.4965792150308,3.3807226270437","6.5077221,3.3618883",2021-07-01,2021-07-01 10:06:34,Thursday,9,1,July,6.496579,3.380723,6.507722,3.361888,2561.0,0,0,2021-07-01


In [31]:
from causalnex.structure import StructureModel
from causalnex.structure.notears import from_pandas

# Convert categorical variables to numeric codes
for col in ['Trip Origin', 'Trip Destination', 'day_of_week', 'month']:
    train_data[col] = pd.Categorical(train_data[col]).codes

# Convert datetime variables to timestamps
for col in ['Trip Start Time', 'Trip End Time', 'Trip Start Date']:
    train_data[col] = pd.to_datetime(train_data[col]).astype(int) / 10**9


In [34]:
train_data.tail(10)

,Trip ID,Trip Origin,Trip Destination,Trip Start Time,Trip End Time,day_of_week,hour_of_day,day_of_month,month,Trip Origin_latitude,Trip Origin_longitude,Trip Destination_latitude,Trip Destination_longitude,trip_duration,is_holiday,is_weekend,Trip Start Date
121,392249,222,216,1.625098e+09,1.625132e+09,1,8,1,0,6.559589,3.385647,6.531492,3.335939,2570.0,0,0,1.625098e+09
499,392904,81,271,1.625098e+09,1.625137e+09,1,10,1,0,6.458720,3.442086,6.576325,3.347018,2684.0,0,0,1.625098e+09
20,392040,126,138,1.625098e+09,1.625148e+09,1,10,1,0,6.500608,3.598194,6.480557,3.280383,13018.0,0,0,1.625098e+09
188,392375,240,224,1.625098e+09,1.625134e+09,1,9,1,0,6.569703,3.262063,6.538651,3.332084,3294.0,0,0,1.625098e+09
71,392148,61,109,1.625098e+09,1.625129e+09,1,8,1,0,6.449082,3.403790,6.459446,3.370149,2183.0,0,0,1.625098e+09
106,392217,166,295,1.625098e+09,1.625132e+09,1,9,1,0,6.524381,3.388800,6.592106,3.338791,2378.0,0,0,1.625098e+09
270,392521,303,278,1.625098e+09,1.625133e+09,1,9,1,0,6.601529,3.363380,6.581541,3.360005,2194.0,0,0,1.625098e+09
348,392660,93,72,1.625098e+09,1.625140e+09,1,9,1,0,6.471358,3.278997,6.447023,3.462735,8250.0,0,0,1.625098e+09
435,392802,237,102,1.625098e+09,1.625147e+09,1,10,1,0,6.568494,3.379576,6.456731,3.530940,12977.0,0,0,1.625098e+09
102,392208,318,164,1.625098e+09,1.625131e+09,1,8,1,0,6.605953,3.386028,6.497595,3.383098,4141.0,0,0,1.625098e+09


In [41]:

from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE

# Learn structure
sm = from_pandas(train_data)


# Visualize the graph
viz = plot_structure(
    sm,
    # graph_attributes={"scale": "0.5"},
    all_node_attributes=NODE_STYLE.WEAK,
    all_edge_attributes=EDGE_STYLE.WEAK)

viz.toggle_physics(False)

filename = "./causal_example.html"
viz.show(filename)
# Image(filename)

./causal_example.html
